# Imports
Imports for the project will be updated as we go along.<br>
The rquired libraries and modules will be imported as needed.

In [ ]:
import pandas as pd

# Data
I have used the data from John Hopkins University which they have provided on GitHub [https://github.com/CSSEGISandData/COVID-19.git]
This data gets updated on constant, frequent intervals (mostly on day-to-day basis).
To access this data, instead of downloading the entire datasets, we can simply link to the files that are available on github and access the data directly without the need for any downloads or local dataset files.

However, we will need to manually change the dates in the data imports.<br>
Just to be safe and avoid any anomalies in the data, We will use data that is two to three days old for reliability.<br>
In the imports for the dataframes from John Hopkins University (ie. latest_data and us_medical_data) we will be using the last updates from two days before.<br>
In the imports for Apple inc. data, (ie. the mobility and analytics from Apple inc.) we will be using data from three days before.

In [ ]:
confirmedDf = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deathsDf = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveriesDf = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latestData = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/09-27-2020.csv')
usMedicalData = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/09-27-2020.csv')
appleMobility = pd.read_csv('https://covid19-static.cdn-apple.com/covid19-mobility-data/2017HotfixDev19/v3/en-us/applemobilitytrends-2020-09-26.csv')

In [ ]:
confirmedDf.head()

In [ ]:
deathsDf.head()

In [ ]:
recoveriesDf.head()

In [ ]:
latestData.head()
# last date and time of updation along with more information such as number of cases, deaths, recoveries, active cases. etc.

In [ ]:
usMedicalData.head()
# US specific data (statewise)

In [ ]:
appleMobility.head()